# Imports

In [34]:
import json
import requests
import pandas as pd

# Activity 1

## 1.1 Sub-activity: Open Data COVID-19 API

### Task 1

In [226]:
# def get_API_data(filters, structure):
#     # API URL
#     url = "https://api.coronavirus.data.gov.uk/v1/data?filters=areaType=" + filters["areaType"] + "&structure=" + json.dumps(structure)
#     # Get data from API
#     response = requests.get(url).json()
#     print(url)

#     # Get cumulative vaccinations by age intervals
#     for data in response["data"]:
#         vaccination_age_dist = {}
#         for vaccination_age in data["vaccination_age"]:
#             vaccination_age_dist[vaccination_age["age"]] = vaccination_age["cumPeopleVaccinatedCompleteByVaccinationDate"]
#         data["vaccination_age"] = vaccination_age_dist
        
#     return response["data"]

def get_vaccination_age_intervals(data):
    vaccination_age_dist = {}
    for vaccination_age in data["vaccination_age"]:
        vaccination_age_dist[vaccination_age["age"]] = vaccination_age["cumPeopleVaccinatedCompleteByVaccinationDate"]
    data["vaccination_age"] = vaccination_age_dist

def get_API_data(filters, structure):
    # API URL
    url = "https://api.coronavirus.data.gov.uk/v1/data?"

    # Get data from region
    if filters["areaType"] == "region":
        region_filters = []
        for i in range(len(filters["areaCode"])):
            filter = 'areaType=' + filters["areaType"] + ';areaCode=' + filters["areaCode"][i]
            region_filters.append(filter)

        responses = []
        for region_filter in region_filters:
            params =  {
                "filters": region_filter,
                "structure": json.dumps(structure)
            }
            
            response = requests.get(url,params = params).json()
            
            # Get cumulative vaccinations by age intervals
            for data in response["data"]:
                get_vaccination_age_intervals(data)
            
            responses.append(response["data"])
        
        return responses

    # Get data from nation
    else:
        params =  {
                "filters": ('areaType=' + filters["areaType"]),
                "structure": json.dumps(structure)
            }

        response = requests.get(url,params = params).json()

        # Get cumulative vaccinations by age intervals
        for data in response["data"]:
            get_vaccination_age_intervals(data)
        
        return response["data"]

### Task 2

In [227]:
nation_filters = {"areaType":"nation"}
region_filters = {
    "areaType":"region",
    "areaCode":[
        "E12000001",
        "E12000002",
        "E12000003",
        "E12000004",
        "E12000005",
        "E12000006",
        "E12000007",
        "E12000008",
        "E12000009"
        ]
    }

structure = {
    "date":"date",
    "name":"areaName",
    "daily_cases":"newCasesBySpecimenDate",
    "cumulative_cases":"cumCasesBySpecimenDate",
    "daily_deaths":"newDeaths28DaysByPublishDate",
    "cumulative_deaths":"cumDeaths28DaysByPublishDate",
    "cumulative_vaccinated":"cumPeopleVaccinatedCompleteByVaccinationDate",
    "vaccination_age":"vaccinationsAgeDemographics"
    }

results_json_national = get_API_data(nation_filters, structure)
results_json_regional = get_API_data(region_filters, structure)

In [228]:
# print(results_json_national)
print(results_json_regional)

[]


## 1.2 Sub-activity: Shaping the COVID data into different dataframes

### Task 3

In [131]:
print(type(len([1,2,3])))

<class 'int'>


In [128]:
from collections import Counter
# a = Counter([data["name"] for data in results_json_national])
b = Counter([data["name"] for data in results_json_regional])
print(b)
# print(a, "\n", b, sep = "")

Counter({'North East': 1005})


### Task 4

In [ ]:
https://api.coronavirus.data.gov.uk/v1/data?filters=areaType=region&structure={"date": "date", "name": "areaName", "daily_cases": "newCasesBySpecimenDate", "cumulative_cases": "cumCasesBySpecimenDate", "daily_deaths": "newDeaths28DaysByPublishDate", "cumulative_deaths": "cumDeaths28DaysByPublishDate", "cumulative_vaccinated": "cumPeopleVaccinatedCompleteByVaccinationDate"}&page=1

### Task 5

### Task 6

### Task 7

### Task 8

### Task 9

### Task 10

### Task 11

### Task 12

### Task 13

### Task 14

## 1.3 Sub-activity: Aggregating, plotting, and analysing

### Task 15

### Task 16

### Task 17

### Task 18

### Task 19

### Task 20

### Task 21

### Task 22

### Task 23

### Task 24

### Task 25

### Task 26

### Task 27

### Task 28